In [1]:
import MySQLdb
import pandas
import re
import itertools
import nltk
# from bs4 import BeautifulSoup

In [44]:
# conn = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugdata')
# cur =conn.cursor()
# sql = '''SELECT bug_id, thetext
# FROM longdescs
# WHERE bug_id = 943195'''
# cur.execute(sql)
# temp = cur.fetchall()
# for item in temp:
#     print len(item[1])
# print temp[4]

In [45]:
def rm_cite(raw_text):
    lst = []
    iscite = False
    lines = raw_text.splitlines()
    for idx,line in enumerate(lines):
        if line.startswith("(In reply to comment"):
            iscite = True
            lst.append(idx)
        elif line.startswith(">"):
            if iscite == True:
                lst.append(idx)
        else:
            iscite = False
        
    return '\n'.join([item[1] for item in filter(lambda x: x[0] not in lst, enumerate(lines))])

In [66]:
from nltk.stem.snowball import SnowballStemmer
# from bs4 import BeautifulSoup

def preprocess_strict( raw_description, output):
    # Function to convert a raw bug description to a list of words or a cleared string, it's "strict" in the sense that it removes all non-letters 
    # Remove markers
    # text = BeautifulSoup(raw_description).get_text() 
    
    # Remove cites
    description = rm_cite(raw_description)
    
    # Remove urls
    text = re.sub("((mailto\:|(news|(ht|f)tp(s?))\://){1}\S+)", " ", description)
    
    # Remove non-letters        
    # letters_only = re.sub("[^a-zA-Z_/\-\.]", " ", description_text)
    letters_only = re.sub("[^a-zA-Z\.]", " ", text)
    letters_only = re.sub("\.(?!((c|h|cpp|py)\s+$))", " ", letters_only)
    
    # Convert to lower case, tokenize
    words = [word for sent in nltk.sent_tokenize(letters_only.lower()) for word in nltk.word_tokenize(sent)]
    
    # Remove stop words
    stopwords = set(nltk.corpus.stopwords.words('english'))
    meaningful_words = [w for w in words if not w in stopwords]
    
    # Stemming
    snowball = SnowballStemmer("english")
    stems = [snowball.stem(w) for w in meaningful_words]
    
    if output == "list":
        return stems
    else:
        return " ".join(stems)

In [74]:
def preprocess_mild( raw_description, output):
    # Function to convert a raw bug description to a list of words or a cleared string, it's "mild" in the sense that it removes words that don't contain letters
    # Remove markers
    # text = BeautifulSoup(raw_description).get_text() 
    
    # Remove cites
    description = rm_cite(raw_description)
    
    # Remove urls
    text = re.sub("((mailto\:|(news|(ht|f)tp(s?))\://){1}\S+)", " ", description)
    text = re.sub("[^\x00-\x7f]", " ", text)
    
    # Convert to lower case, tokenize
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    
    # Remove stop words
    stopwords = set(nltk.corpus.stopwords.words('english'))
    meaningful_tokens = [token for token in filtered_tokens if not token in stopwords]
    
    # Stemming
    snowball = SnowballStemmer("english")
    stems = [snowball.stem(t) for t in meaningful_tokens]
    
    if output == "list":
        return stems
    else:
        return " ".join(stems)

In [49]:
conn_data = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugdata')
cur_data =conn_data.cursor()

sql = '''SELECT bug_id, bug_severity, short_desc, host_op_sys, guest_op_sys, product_id, category_id, component_id
FROM bugs,profiles 
WHERE bugs.assigned_to=profiles.userid and profiles.login_name in ('hfu','letian','vbhakta','weili','nmukuri','zhoum','hxie','shiyaoy','shanpeic','souravk','vaibhavk','fangchiw','gengshengl')'''

cur_data.execute(sql)
bugs_cpdplatf = cur_data.fetchall()

In [75]:
import itertools

bugdata = []

conn_data = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugdata')
cur_data =conn_data.cursor()

for item in bugs_cpdplatf:
    sql_ld = '''SELECT bug_id, thetext
    FROM longdescs
    WHERE bug_id = (%s)''' % str(item[0])
    
    df = pandas.io.sql.read_sql(sql_ld, conn_data)
    
    keywords = preprocess_strict(item[2])
    
    text = list(df['thetext'].map(lambda x: preprocess_mild(x,"list")))
    
    def judge(wordlist):       
        if (len(wordlist) > 10 and len(set(keywords) & set(wordlist)) > 0 ) or len(wordlist) > 30:
            return True
        else:
            return False
    
    long_desc = list(itertools.chain(*filter(judge, text)))
    
    bugdata.append((str(item[0]), str(item[1]), " ".join(keywords), str(item[3]), str(item[4]), str(item[5]), str(item[6]), str(item[7]), " ".join(long_desc)))
    


In [76]:
conn_feature = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugfeature')
cur_feature =conn_feature.cursor()

sql_ins = '''INSERT into bugs_cpdplatform
VAlUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'''
    
cur_feature.executemany(sql_ins,bugdata)
conn_feature.commit()

In [60]:
# conn_data = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugdata')
# cur_data =conn_data.cursor()

# sql_ld = '''SELECT bug_id, thetext
# FROM longdescs
# WHERE bug_id = 1297787'''
    
# df = pandas.io.sql.read_sql(sql_ld, conn_data)

# for idx, row in df.iterrows():
#     print idx,'\n', "*****", '\n', row['thetext']

# text = list(df['thetext'].map(lambda x: preprocess_strict(x,"list")))

# for item in text:
#     print len(item)

0 
***** 
*** This child bug 1297787 has been created from base bug 1293636.  Please keep the conversation on the base bug when appropriate. ***
1 
***** 
Thanks fangchi for taking it.
2 
***** 
Per base bug, this issue should be handled in hosted products.
3 
***** 
remove keyword triage-accepted.
11
3
7
4


In [17]:
conn_feature = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugfeature')
cur_feature =conn_feature.cursor()

sql = '''SELECT bug_id
FROM bugs_cpdplatform'''
cur_feature.execute(sql)
bugid = cur_feature.fetchall()
bugid = [str(item[0]) for item in bugid]

In [18]:
# bugid = [str(item) for item in bugid]
# print bugid
# print ",".join(bugid)

In [23]:
conn_data = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugdata')
cur_data =conn_data.cursor()

sql = '''SELECT *
FROM duplicates
WHERE dupe_of in (%s)
and dupe in (%s)''' %(",".join(bugid),",".join(bugid))

cur_data.execute(sql)
dups = cur_data.fetchall() 

((140193, 140328), (158787, 158778), (158787, 158781), (158787, 158789), (164981, 171016), (199825, 183074), (156960, 240657), (252326, 256727), (251889, 256851), (274057, 274061), (289577, 290423), (259769, 290925), (296424, 299452), (310456, 313360), (350655, 350697), (504247, 353002), (354998, 355000), (360704, 360705), (372652, 375843), (354710, 379729), (415862, 412679), (530331, 425069), (448751, 447305), (482099, 450273), (424730, 453885), (425054, 453886), (505405, 538482), (542546, 560250), (604024, 601323), (601789, 601480), (601598, 601597), (600735, 604665), (609458, 609800), (614338, 614337), (704605, 624350), (628759, 629845), (636113, 661557), (721980, 692089), (701177, 701180), (701178, 701181), (691280, 704440), (654800, 714889), (721980, 720525), (714325, 720873), (704605, 720942), (907080, 750471), (753475, 753961), (746939, 753981), (746941, 753985), (753475, 783137), (714814, 787325), (815930, 816106), (815930, 816168), (794510, 821121), (1074199, 824880), (790790,

In [24]:
conn_feature = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugfeature')
cur_feature =conn_feature.cursor()

sql_ins = '''INSERT into dups_cpdplatform
VAlUES (%s, %s)'''
    
cur_feature.executemany(sql_ins,dups)
conn_feature.commit()

In [26]:
print len(bugid), len(dups)

4013 239
